In [63]:
import os
import math
import jieba as jb
import collections

In [64]:
# 读取所有的数据文件
all_files = []
listFile = os.walk(r"DATA")
for dirPath, dirName, fileName in listFile:
    for names in fileName:
        all_files.append(os.path.join(dirPath, names))
print(all_files)

['DATA/时政/371306.txt', 'DATA/时政/392336.txt', 'DATA/时政/360792.txt', 'DATA/时政/350276.txt', 'DATA/时政/392337.txt', 'DATA/时政/371307.txt', 'DATA/时政/381821.txt', 'DATA/时政/381822.txt', 'DATA/时政/350277.txt', 'DATA/时政/360791.txt', 'DATA/教育/291448.txt', 'DATA/教育/291449.txt', 'DATA/教育/298438.txt', 'DATA/教育/298439.txt', 'DATA/教育/319409.txt', 'DATA/教育/319408.txt', 'DATA/教育/312418.txt', 'DATA/教育/305428.txt', 'DATA/教育/312419.txt', 'DATA/教育/305429.txt', 'DATA/游戏/422679.txt', 'DATA/游戏/410489.txt', 'DATA/游戏/414553.txt', 'DATA/游戏/418615.txt', 'DATA/游戏/418616.txt', 'DATA/游戏/414552.txt', 'DATA/游戏/426742.txt', 'DATA/游戏/410490.txt', 'DATA/游戏/426741.txt', 'DATA/游戏/422678.txt', 'DATA/体育/66160.txt', 'DATA/体育/4923.txt', 'DATA/体育/49240.txt', 'DATA/体育/32318.txt', 'DATA/体育/15399.txt', 'DATA/体育/83082.txt', 'DATA/体育/116920.txt', 'DATA/体育/15398.txt', 'DATA/体育/116921.txt', 'DATA/体育/83081.txt', 'DATA/体育/66161.txt', 'DATA/体育/32319.txt', 'DATA/科技/535959.txt', 'DATA/科技/590268.txt', 'DATA/科技/617423.txt', 'DATA/科技/644578.txt'

In [65]:
# 获取停用词列表
def get_stopword_list(file):
    with open(file, 'r', encoding='utf-8') as f:
        stopword_list = [word.strip('\n') for word in f.readlines()]
    return stopword_list
print(get_stopword_list("stopwords.txt"))

['———', '》），', '）÷（１－', '”，', '）、', '＝（', ':', '→', '℃ ', '&', '*', '一一', '~~~~', '’', '. ', '『', '.一', './', '-- ', '』', '＝″', '【', '［＊］', '｝＞', '［⑤］］', '［①Ｄ］', 'ｃ］', 'ｎｇ昉', '＊', '//', '［', '］', '［②ｅ］', '［②ｇ］', '＝｛', '}', '，也 ', '‘', 'Ａ', '［①⑥］', '［②Ｂ］ ', '［①ａ］', '［④ａ］', '［①③］', '［③ｈ］', '③］', '１． ', '－－ ', '［②ｂ］', '’‘ ', '××× ', '［①⑧］', '０：２ ', '＝［', '［⑤ｂ］', '［②ｃ］ ', '［④ｂ］', '［②③］', '［③ａ］', '［④ｃ］', '［①⑤］', '［①⑦］', '［①ｇ］', '∈［ ', '［①⑨］', '［①④］', '［①ｃ］', '［②ｆ］', '［②⑧］', '［②①］', '［①Ｃ］', '［③ｃ］', '［③ｇ］', '［②⑤］', '［②②］', '一.', '［①ｈ］', '.数', '［］', '［①Ｂ］', '数/', '［①ｉ］', '［③ｅ］', '［①①］', '［④ｄ］', '［④ｅ］', '［③ｂ］', '［⑤ａ］', '［①Ａ］', '［②⑧］', '［②⑦］', '［①ｄ］', '［②ｊ］', '〕〔', '］［', '://', '′∈', '［②④', '［⑤ｅ］', '１２％', 'ｂ］', '...', '...................', '…………………………………………………③', 'ＺＸＦＩＴＬ', '［③Ｆ］', '」', '［①ｏ］', '］∧′＝［ ', '∪φ∈', '′｜', '｛－', '②ｃ', '｝', '［③①］', 'Ｒ．Ｌ．', '［①Ｅ］', 'Ψ', '－［＊］－', '↑', '.日 ', '［②ｄ］', '［②', '［②⑦］', '［②②］', '［③ｅ］', '［①ｉ］', '［①Ｂ］', '［①ｈ］', '［①ｄ］', '［①ｇ］', '［①②］', '［②ａ］', 'ｆ］', '［⑩］', 'ａ］', '

In [66]:
# 把数据洗一洗，去掉停用词，\n,\t等，仅保留关键部分
stop_words = get_stopword_list("stopwords.txt")
stop_words.extend(['\n','\t','\u3000','★','\xa0','…',' '])

# 因为这里只做一个demo，所以所有的内容保存在内存里吧，大家的内存应该都够大
clean_data = []
for file_path in all_files:
    with open(file_path, mode="r", encoding="utf-8") as f:
        text = f.read()
        word_list = list(jb.cut_for_search(text))
        word_list = [w for w in word_list if w not in stop_words]
#         print(word_list)
        clean_data.append(word_list)
    
assert len(clean_data) == len(all_files)

In [67]:
# 记录总的文件数
cnt = len(all_files)
#  倒排索引
reverted_idx = dict()
# 记录文档向量的模
doc_len = dict()

# 构建最基础的倒排索引
def constructIndex(data, doc_id):
    for str in data:
        if (reverted_idx.__contains__(str)):
            if doc_id in reverted_idx[str]:
                reverted_idx[str][doc_id] += 1
            else:
                reverted_idx[str][doc_id] = 1
        else:
            reverted_idx[str] = {doc_id: 1}

# 稍微整理一下这些index,并且计算出文档向量的长度
def sortIndex():
    for k, v in reverted_idx.items():
        curr = [len(reverted_idx[k])]
        curr.extend(sorted([[x, y] for x, y in reverted_idx[k].items()], key=lambda z: z[0]))
        reverted_idx[k] = curr

    for k, v in reverted_idx.items():
        idf = math.log(cnt / reverted_idx[k][0], 10)
        for i in range(len(reverted_idx[k])):
            if i == 0:
                continue  
            tf = (1 + math.log(reverted_idx[k][i][1], 10))
            doc_len[reverted_idx[k][i][0]] = doc_len.get(
                reverted_idx[k][i][0], 0) + idf * tf * idf * tf
# 构建倒排索引
for i in range(cnt):
    constructIndex(clean_data[i], i)
# 处理倒排索引
sortIndex()
reverted_idx

{'巴方': [1, [0, 1]],
 '指责': [2, [0, 2], [7, 1]],
 '希拉': [1, [0, 4]],
 '拉里': [1, [0, 4]],
 '希拉里': [1, [0, 4]],
 '宽恕': [1, [0, 3]],
 '以色列': [3, [0, 8], [2, 5], [6, 9]],
 '破坏': [1, [0, 2]],
 '和平': [2, [0, 2], [7, 2]],
 '中新': [4, [0, 1], [2, 1], [4, 1], [8, 1]],
 '新网': [4, [0, 1], [2, 1], [4, 1], [8, 1]],
 '中新网': [4, [0, 1], [2, 1], [4, 1], [8, 1]],
 '11': [6, [0, 1], [5, 1], [10, 1], [11, 3], [19, 2], [29, 2]],
 '月': [31,
  [0, 1],
  [1, 1],
  [2, 3],
  [3, 3],
  [4, 1],
  [5, 1],
  [6, 1],
  [7, 4],
  [8, 1],
  [9, 2],
  [10, 1],
  [11, 1],
  [19, 2],
  [23, 2],
  [25, 2],
  [27, 1],
  [28, 1],
  [29, 5],
  [30, 1],
  [33, 1],
  [36, 1],
  [37, 1],
  [38, 1],
  [39, 1],
  [40, 1],
  [42, 1],
  [43, 3],
  [45, 1],
  [49, 3],
  [50, 1],
  [51, 1]],
 '2': [17,
  [0, 1],
  [2, 2],
  [3, 1],
  [5, 1],
  [7, 2],
  [9, 1],
  [11, 3],
  [17, 2],
  [19, 2],
  [21, 1],
  [22, 3],
  [23, 1],
  [25, 2],
  [32, 1],
  [36, 1],
  [39, 1],
  [44, 3]],
 '日电': [8, [0, 1], [1, 1], [2, 1], [3, 1], [4, 1], [7

In [68]:
def search(query):
    docToBeProcess = []
    for i in range(len(query)):
        docToBeProcess.append(reverted_idx.get(query[i], []))
    
    # counter里面保存着所有这条query涉及到的doc_id
    counter = set()
    for i in range(len(docToBeProcess)):
        for k in range(len(docToBeProcess[i])):
            if k == 0:
                continue
            else:
                counter.add(docToBeProcess[i][k][0])
    # 保存文档的分数
    grade = {}
    for i in counter:
        grade[i] = 0
    
    #Counter({'想': 1, '看看': 1, '只': 1, '小鸟': 1})
    querycount = collections.Counter(query)

    for i in range(len(docToBeProcess)):
        for k in range(len(docToBeProcess[i])):
            if k == 0:
                continue
            idf = math.log(cnt / docToBeProcess[i][0], 10)
            doc_vector = idf * (1 + math.log(docToBeProcess[i][k][1], 10))
            query_vector = (1 + math.log(querycount[query[i]], 10)) * idf
            grade[docToBeProcess[i][k][0]] += doc_vector * query_vector
    
    # doc的排名
    rank = []
    for k, v in grade.items():
        grade[k] = grade[k] / math.sqrt(doc_len[k])
        rank.append((k, grade[k]))

    rank = [x[0] for x in sorted(rank, key=lambda x: x[1], reverse=True)]

    if len(rank)>3:
        return rank[:3]
    else:
        return rank

In [71]:
query = input("请输入你想要查询的内容")
print()
query = list(jb.cut_for_search(query))
query = [w for w in query if w not in stop_words]
# 返回前三(或唯一一个搜索结果),但是我们只展示最相关的那个文章
doc_idx = search(query)
if len(doc_idx) == 0:
    print("很抱歉，搜索不到相关内容(例子：可以搜索 “我认为中东立场不好” 试试！)")
else:
    with open(all_files[doc_idx[0]], mode="r", encoding="utf-8") as f:
        print(f.read())

请输入你想要查询的内容运动生理学

09年成考北京250余名运动员参加单独招生考试
　　2009年国家体育总局成人高等教育运动训练专业单独招生考试于11月7日、8日举行，北京考区共250多名考生参加。
　　记者在北京体育大学考点看到，设有考场的楼层入口处有保安人员现场执勤。所有考生、监考人员凭证件才可入内。楼道内，有巡视人员检查；考场内，数十名运动员考生在专心答题，两名监考人员认真履行着监督职责。
　　今年成考中，北京地区只有北京体育大学通过单考单招方式招收运动员考生。该考点负责人介绍，全国共有1000余名考生报考，其中北京地区有250余名。考生报考专业均为运动训练专业，有高起专和专升本两个学习层次。考生根据学习层次选择考试科目。其中专升本层次为运动生理学、运动心理学、运动训练学和综合能力4科，高起专层次为语文、数学、外语和政治4科。该考点参加考试的考生中，200余名为高起专层次，50余名为专升本层次。
　　据了解，国家体育总局的成人高等教育运动训练专业单考单招，一般在全国成人高校统一招生考试后举行。优秀运动队(省、自治区、直辖市、计划单列市、大军区、各行业体协)的教练员、运动员以及运动学校和业余体校的教练员均可参加。该招生专业只设置了高起专和专升本两个层次。
　　相关链接
　　单考单招考生可不用参加全国统考。单考单招的学校单列招生计划，考试单独命题、单独录取。目前北京地区开展的成人高考单考单招项目有两类，一类是北京联合大学特殊教育学院针对残疾考生进行的单考单招，另一类是国家体育总局对运动员进行的单考单招。参加单考单招的考生，可享受一定的学费、录取等优惠政策。单考单招一般有固定的专业，同时对报考人员身份有一定限制。


